In [1]:
!pip install patool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.7 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification, DataCollatorWithPadding
import logging
import patoolib

from custom_model import NERModel
import create_dataset_2_0

In [3]:
patoolib.extract_archive("formatted.rar", outdir=".")

INFO patool: Extracting formatted.rar ...
INFO:patool:Extracting formatted.rar ...
INFO patool: running /usr/bin/unrar x -- /content/formatted.rar
INFO:patool:running /usr/bin/unrar x -- /content/formatted.rar
INFO patool:     with cwd='.', input=''
INFO:patool:    with cwd='.', input=''
INFO patool: ... formatted.rar extracted to `.'.
INFO:patool:... formatted.rar extracted to `.'.


'.'

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
logging.info(f"Running on {device}")
print(device)

cuda


In [5]:
PRECISION_RECALL_OFFSET = 0.1
MAX_LEN = 512
MAX_EPOCHS = 20
BATCH_SIZE = 128

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model_orig_backbone = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

model = NERModel(backbone=model_orig_backbone)

for param in model.backbone.parameters():
    param.requires_grad = False

model.to(device)

NERModel(
  (backbone): DistilBertForMaskedLM(
    (activation): GELUActivation()
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [8]:
train_dataset, val_dataset, neg_pos_ratio = create_dataset_2_0.load_dataset(
    json_path='formatted',
    tokenizer=tokenizer,
    max_len=MAX_LEN,
    split_percent=0.95
)

In [9]:
# construct training / validation loaders with dynamic batching
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator
)

In [11]:
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor(
    [PRECISION_RECALL_OFFSET * neg_pos_ratio]).to(device))

optimizer = torch.optim.AdamW(params=model.parameters(), lr=3e-4)

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LinearLR.html
scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=1, end_factor=0.01, total_iters=MAX_EPOCHS)

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#     optimizer, T_max=MAX_EPOCHS, last_epoch=48, eta_min=3e-6
# )

In [ ]:
# for early stopping
min_validation_loss = None
patience = 3
trigger_times = 0
last_val_loss = 100

# I would've used a Trainer
for epoch in range(MAX_EPOCHS):
    training_loss = 0
    model.train()

    for batch_id, data in enumerate(train_loader):

        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        outputs = model(ids, mask)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        training_loss += loss.item()

    logging.info(
        f"Training loss @{epoch + 1} : {training_loss * BATCH_SIZE / len(train_loader)}")
    print(
        f"Training loss @epoch {epoch + 1} : {training_loss * BATCH_SIZE / len(train_loader)}")

    validation_loss = 0
    tps, tns, fps, fns = 0, 0, 0, 0
    model.eval()

    with torch.no_grad():
        for batch_id, data in enumerate(val_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask)
            loss = criterion(outputs, targets)

            outputs = torch.sigmoid(outputs)

            for i in range(outputs.shape[0]):
                if outputs[i] < 0.5 and targets[i] < 0.5:
                    tns += 1

                if outputs[i] < 0.5 < targets[i]:
                    fns += 1

                if outputs[i] > 0.5 > targets[i]:
                    fps += 1

                if outputs[i] > 0.5 and targets[i] > 0.5:
                    tps += 1

            validation_loss += loss.item()

        val_loss_epoch = validation_loss * BATCH_SIZE / len(val_loader)
        logging.info(
            f"Validation loss @{epoch + 1} : {val_loss_epoch}")
        print(
            f"Validation loss @epoch {epoch + 1} : {val_loss_epoch}")

        # https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch
        if min_validation_loss is None or validation_loss < min_validation_loss:
          min_validation_loss = validation_loss
          trigger_times = 0

          logging.info(
              f"Saving new model > checkpoints_data_aug/{min_validation_loss}.dat")
          print(
              f"Saving new model > checkpoints_data_aug/{min_validation_loss}.dat")
          torch.save(model.state_dict(),
                      f"checkpoints_data_aug/{min_validation_loss}.dat")
        elif min_validation_loss is not None and validation_loss > min_validation_loss:
          trigger_times += 1

          if trigger_times >= patience:
            break
        logging.info(f'TP:{tps} TN:{tns} FP:{fps} FN:{fns}')
        print(f'TP:{tps} TN:{tns} FP:{fps} FN:{fns}')

        scheduler.step()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training loss @epoch 1 : 4.452686204746899
Validation loss @epoch 1 : 4.551408375357534
Saving new model > checkpoints_data_aug/3.2357668918557465.dat
TP:1014 TN:10489 FP:20 FN:122
Training loss @epoch 2 : 2.8077849726662163
Validation loss @epoch 2 : 2.8130823974753474
Saving new model > checkpoints_data_aug/1.9999257669551298.dat
TP:1091 TN:10459 FP:48 FN:47


In [ ]:
!cp `ls -t *.dat ./checkpoints_data_aug | tail -2` ./download

In [ ]:
!zip -r ./model_distilbert_ner.zip ./download

In [ ]:
from google.colab import files # For downloading the model
files.download('./model_distilbert_ner.zip')